Read a markdown file and number the headdings. Assumes that all headdings are marked as lines beginning with one or more # characters per markdown syntax.

In [ ]:
import argparse
import re
import sys

class NumberedHeadding:
    
    MAX_LEVELS = 3
    
    def __init__(self):
        self.numbered = [None] * NumberedHeadding.MAX_LEVELS
        self.level = 0
        self.numbered[self.level] = 0

    def more_levels(self):
        return self.level + 1 < NumberedHeadding.MAX_LEVELS

    def less_levels(self):
        return 0 < self.level
    
    def indent(self):
        if self.more_levels():
            self.level += 1
            self.numbered[self.level] = 1
            
    def outdent(self):
        if self.less_levels():
            self.numbered[self.level] = None
            self.level -= 1
            self.increment()
            
    def increment(self):
        self.numbered[self.level] += 1
    
    def set_level(self, new_level):
        if new_level == self.level:
            self.increment()
        else:
            for i in xrange(self.level, new_level):
                self.indent()
            for i in xrange(self.level, new_level, -1):
                self.outdent()
            
    def markdown_to_headding(self, markdown_string):
        m = re.compile(
            '^(?P<hash>#+)'      # Markdown headding number
            '\s*[\d\.]*\s*'      # Headding number
            '(?P<remainder>.*)'  # Headding text
        )
        if m.match(markdown_string):        
            hashes = m.match(markdown_string).group('hash')
            remainder = m.match(markdown_string).group('remainder')
            self.set_level(len(hashes)-1)
            return hashes + ' ' + self.headding() + ' ' + remainder
        else:
            return markdown_string
            
    def headding(self):
        # ignore '#', used as document title
        return '.'.join(str(x) for x in self.numbered[1:self.level+1])

class ConfigureProgram:
    def __init__(self):
        parser = argparse.ArgumentParser(description="Create numbered headdings in md files")
    
        # --argument value
        # value stored in args.argument
        parser.add_argument("--in", dest='file_in', action="store", help="Markdown file to process")
        parser.add_argument("--out", dest='file_out', action="store", help="Output file to write")

        # results of parsing arguments
        self.args = parser.parse_args()
        

In [ ]:
# simulate the command line

markdown_file = 'input.md'
output_file = 'output.md'

sys.argv = ['PROGRAM', '--in', markdown_file, '--out', output_file]

In [ ]:
if __name__ == "__main__":
    config = ConfigureProgram()
    
    headdings = NumberedHeadding()

    fh = open(config.args.file_in, 'r')
    output = open(config.args.file_out, 'w')

    for line in fh:
        output.write(h.markdown_to_headding(line))
        
    output.close()